In [1]:
pip install tensorflow keras albumentations opencv-python numpy matplotlib --user

Note: you may need to restart the kernel to use updated packages.


In [17]:
import os
import numpy as np
import cv2
import tensorflow as tf 
import albumentations as A
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report
import tensorflow.keras.backend as K


In [38]:
class TeethDataGenerator(Sequence):
    def __init__(self, image_paths, mask_paths, batch_size=8, image_size=(256, 256), augment=True, class_weighting=True, **kwargs):
        super().__init__(**kwargs)
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.batch_size = batch_size
        self.image_size = image_size
        self.augment = augment
        self.class_weighting = class_weighting
        self.indexes = np.arange(len(self.image_paths))
        self.on_epoch_end()
        
        self.augmentation = A.Compose([
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.RandomBrightnessContrast(p=0.4),
            A.GaussianBlur(p=0.3),
            A.ElasticTransform(alpha=2, sigma=50, alpha_affine=50, p=0.5),
            A.RandomCrop(height=256, width=256, p=0.7)
        ])

    def __len__(self):
        return int(np.floor(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch_images = [self.image_paths[k] for k in indexes]
        batch_masks = [self.mask_paths[k] for k in indexes]
        images, masks = self.__data_generation(batch_images, batch_masks)
        return np.array(images), np.array(masks)

    def __data_generation(self, batch_images, batch_masks):
        images, masks = [], [] 
        for img_path, mask_path in zip(batch_images, batch_masks):
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, self.image_size) / 255.0  # Normalize image

            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, self.image_size) / 255.0  # Normalize mask

            # Convert mask to binary: Thresholding (values >= 0.5 become 1, else 0)
            mask = np.where(mask >= 0.5, 1, 0).astype(np.uint8)

            # Mask Check: Ensure only 0 & 1 values
            unique_values = np.unique(mask)
            #print(f"Mask unique values for {mask_path}: {unique_values}")  # Debugging output
            if not np.all(np.isin(unique_values, [0, 1])):
                raise ValueError(f"ERROR: Unexpected mask values in {mask_path}: {unique_values}. Expected only {0, 1}.")


            if self.augment:
                augmented = self.augmentation(image=img, mask=mask)
                img, mask = augmented["image"], augmented["mask"]

            images.append(np.expand_dims(img, axis=-1))
            masks.append(np.expand_dims(mask, axis=-1))

        images, masks = np.array(images), np.array(masks)

        if self.class_weighting:
            mask_weights = np.where(masks > 0.5, 2.0, 1.0)  # Give more weight to teeth pixels
            masks = masks * mask_weights

        return images, masks

    def on_epoch_end(self):
        np.random.shuffle(self.indexes)


In [39]:
def conv_block(x, num_filters):
    x = layers.Conv2D(num_filters, (3, 3), padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    
    x = layers.Conv2D(num_filters, (3, 3), padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    
    return x

def upsample_block(x, skip, num_filters):
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Concatenate()([x, skip])
    x = conv_block(x, num_filters)
    return x

from tensorflow.keras import layers

def build_unetpp(input_shape=(256, 256, 1)):
    inputs = layers.Input(input_shape)

    c1 = layers.Conv2D(32, (3, 3), padding="same", activation="relu")(inputs)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(64, (3, 3), padding="same", activation="relu")(p1)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(128, (3, 3), padding="same", activation="relu")(p2)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(256, (3, 3), padding="same", activation="relu")(p3)
    p4 = layers.MaxPooling2D((2, 2))(c4)

    c5 = layers.Conv2D(512, (3, 3), padding="same", activation="relu")(p4)

    u4 = layers.UpSampling2D((2,2))(c5)
    u3 = layers.UpSampling2D((2,2))(u4)
    u2 = layers.UpSampling2D((2,2))(u3)
    u1 = layers.UpSampling2D((2,2))(u2)

    outputs = layers.Conv2D(1, (1, 1), activation="sigmoid")(u1)

    return keras.models.Model(inputs, outputs)



In [40]:
# initialize data generators
train_gen = TeethDataGenerator(train_images, train_masks, batch_size=8, augment=True)
val_gen = TeethDataGenerator(val_images, val_masks, batch_size=8, augment=False)

C:\Users\FAST\AppData\Local\Temp\ipykernel_9440\1125605907.py:18: UserWarning: Argument(s) 'alpha_affine' are not valid for transform ElasticTransform
  A.ElasticTransform(alpha=2, sigma=50, alpha_affine=50, p=0.5),


In [41]:
def dice_coefficient(y_true, y_pred, smooth=1e-6):
    y_true_f = K.flatten(tf.cast(y_true, dtype=tf.float32))  # Ensure float32
    y_pred_f = K.flatten(tf.cast(y_pred, dtype=tf.float32))  # Ensure float32
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)

def focal_loss(y_true, y_pred, gamma=2.0, alpha=0.9):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    bce = K.binary_crossentropy(y_true, y_pred)
    p_t = y_true * y_pred + (1 - y_true) * (1 - y_pred)
    focal = alpha * K.pow((1 - p_t), gamma) * bce
    return K.mean(focal)

def combined_loss(y_true, y_pred):
    return 0.5 * dice_loss(y_true, y_pred) + 0.5 * focal_loss(y_true, y_pred)


In [43]:
# Define Dataset Paths
dataset_path = r"C:\Users\FAST\Desktop\FYP_work-Dental\FYP_work\DecayDataSrc\Teeth_Dataset\augmented"
image_dir = os.path.join(dataset_path, "Images1")
mask_dir = os.path.join(dataset_path, "Masks1")

# Get Sorted Image and Mask Paths
image_paths = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.png')])
mask_paths = sorted([os.path.join(mask_dir, f) for f in os.listdir(mask_dir) if f.endswith('.png')])

# Ensure Correct Image-Mask Pairing
assert len(image_paths) == len(mask_paths), "Mismatch in number of images and masks!"

# Split Dataset: 80% Train, 20% Validation
split = int(0.8 * len(image_paths))
train_images, val_images = image_paths[:split], image_paths[split:]
train_masks, val_masks = mask_paths[:split], mask_paths[split:]

print(f"Training Samples: {len(train_images)}, Validation Samples: {len(val_images)}")

Training Samples: 464, Validation Samples: 116


In [44]:
model = build_unetpp(input_shape=(256, 256, 1))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
              loss=combined_loss, 
              metrics=[dice_coefficient])
checkpoint_path = "unetpp_best_model_2.keras"
callbacks = [
    ModelCheckpoint(filepath=checkpoint_path, monitor='val_dice_coefficient', save_best_only=True, mode='max', verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
]

history = model.fit(train_gen, validation_data=val_gen, epochs=80, callbacks=callbacks)

# Save Final Model
model.save("unetpp_final_trained_2.h5")


Epoch 1/80
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 572ms/step - dice_coefficient: 0.4927 - loss: 0.3327
Epoch 1: val_dice_coefficient improved from -inf to 0.56732, saving model to unetpp_best_model_2.keras
58/58 ━━━━━━━━━━━━━━━━━━━━ 39s 631ms/step - dice_coefficient: 0.4932 - loss: 0.3324 - val_dice_coefficient: 0.5673 - val_loss: 0.2872 - learning_rate: 1.0000e-04
Epoch 2/80
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 570ms/step - dice_coefficient: 0.6423 - loss: 0.2544
Epoch 2: val_dice_coefficient improved from 0.56732 to 0.79035, saving model to unetpp_best_model_2.keras
58/58 ━━━━━━━━━━━━━━━━━━━━ 36s 622ms/step - dice_coefficient: 0.6432 - loss: 0.2540 - val_dice_coefficient: 0.7903 - val_loss: 0.2432 - learning_rate: 1.0000e-04
Epoch 3/80
58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 570ms/step - dice_coefficient: 0.8098 - loss: 0.1617
Epoch 3: val_dice_coefficient improved from 0.79035 to 0.85332, saving model to unetpp_best_model_2.keras
58/58 ━━━━━━━━━━━━━━━━━━━━ 36s 621ms/step - dice_coefficient: 0.8101 - loss: 0.

KeyboardInterrupt: 

In [45]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report

# ✅ Load the trained model
model = load_model("unetpp_best_model_2.keras", custom_objects={
    "dice_coefficient": dice_coefficient,
    "dice_loss": dice_loss,
    "combined_loss": combined_loss
})
print("✅ Model loaded successfully!")

# ✅ Function to preprocess images
def preprocess_image(image_path, target_size=(256, 256)):
    img = tf.keras.preprocessing.image.load_img(image_path, color_mode="grayscale", target_size=target_size)
    img = tf.keras.preprocessing.image.img_to_array(img) / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# ✅ Function to preprocess masks (convert to binary labels)
def preprocess_mask(mask_path, target_size=(256, 256)):
    mask = tf.keras.preprocessing.image.load_img(mask_path, color_mode="grayscale", target_size=target_size)
    mask = tf.keras.preprocessing.image.img_to_array(mask) / 255.0  # Normalize
    mask = (mask > 0.5).astype(int)  # Convert to binary mask (0 or 1)
    return mask.flatten()  # Flatten mask to 1D array

# ✅ Load validation images and masks
X_val = np.vstack([preprocess_image(img) for img in val_images])  # Validation images
y_val = np.concatenate([preprocess_mask(mask) for mask in val_masks])  # Validation masks

print("✅ Validation data prepared successfully!")

# ✅ Get predictions
y_pred = model.predict(X_val, batch_size=8)
y_pred = (y_pred > 0.5).astype(int)  # Convert to binary (0 or 1)
y_pred = y_pred.flatten()  # Flatten predictions

print("✅ Predictions generated!")

# ✅ Print classification report
print("\n📊 Classification Report:")
print(classification_report(y_val, y_pred, target_names=["Background", "Teeth"]))


✅ Model loaded successfully!
✅ Validation data prepared successfully!
15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 200ms/step
✅ Predictions generated!

📊 Classification Report:
              precision    recall  f1-score   support

  Background       0.96      0.71      0.82   6130678
       Teeth       0.42      0.88      0.57   1471498

    accuracy                           0.75   7602176
   macro avg       0.69      0.80      0.70   7602176
weighted avg       0.86      0.75      0.77   7602176

